In [10]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

data=pd.read_excel("赛题1数据材料\赛题1数据集\融资信息.xlsx")
data=data.replace('未披露',np.nan)
data.columns=['企业编号', '融资日期', '轮次', '$投资金额']
data

,企业编号,融资日期,轮次,$投资金额
0,1001,1999-11-10,IPO,NaN
1,1002,2003-04-28,IPO,NaN
2,1002,2017-12-21,战略投资,NaN
3,1003,1999-07-27,IPO,NaN
4,1004,1999-03-12,IPO,NaN
...,...,...,...,...
4521,3998,2011-04-26,IPO,NaN
4522,3999,2011-04-26,IPO,NaN
4523,4000,2007-11-27,A轮,585万人民币
4524,4000,2011-01-01,B轮,NaN


In [11]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

df=data.set_index("企业编号")
df1 = df[['融资日期']].applymap(to_day).groupby('企业编号').mean().T
df1 = df1.reindex(columns=list(range(1001, 4001)))
df1

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
融资日期,-7055.0,-3114.5,-7161.0,-7298.0,-6886.0,-7685.0,-6570.0,-5603.0,-6741.0,-2909.5,...,-2287.5,-2937.0,-2881.0,-4272.6,-3328.666667,-2875.0,-1604.666667,-2870.0,-2870.0,-3320.666667


In [14]:
data=data.set_index("轮次").loc[["IPO","战略投资","A轮","上市","主板定向增发",
                          "B轮","天使轮","天使轮","股权转让","被收购","C轮","新三板"],:].reset_index()
for i in range(len(data)):
    if ( type(data.iloc[i,3])==np.str and data.iloc[i,3][0]!="数"):
        if(data.iloc[i,3][-1]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-1])
        elif(data.iloc[i,3][-2]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-2])
        elif(data.iloc[i,3][-3]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-3])
        elif(data.iloc[i,3][-4]=="万"):
             data.iloc[i,3]=float(data.iloc[i,3][:-4])
    elif (type(data.iloc[i,3])==np.str and data.iloc[i,3][0]=="数"):
            data.iloc[i,3]=np.nan
data.columns=["轮次","企业编号","融资日期","$投资金额"]
data

,轮次,企业编号,融资日期,$投资金额
0,IPO,1001,1999-11-10,NaN
1,IPO,1002,2003-04-28,NaN
2,IPO,1003,1999-07-27,NaN
3,IPO,1004,1999-03-12,NaN
4,IPO,1005,2000-04-27,NaN
...,...,...,...,...
4489,新三板,3782,2015-11-19,NaN
4490,新三板,3784,2014-01-24,NaN
4491,新三板,3793,2015-01-27,NaN
4492,新三板,3794,2015-11-12,NaN


In [15]:
data=data.drop(columns="融资日期").set_index('企业编号')
result=data.groupby(['企业编号','轮次']).sum().unstack().T.reindex(columns=list(range(1001, 4001)))
result

企业编号        1001  1002  1003  1004  1005  1006  1007      1008  1009  \
      轮次                                                               
$投资金额 A轮     NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN   NaN   
      B轮     NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN   NaN   
      C轮     NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN   NaN   
      IPO    0.0   0.0   0.0   0.0   0.0   0.0   0.0       NaN   0.0   
      上市     NaN   NaN   NaN   NaN   NaN   NaN   NaN  278250.0   NaN   
...          ...   ...   ...   ...   ...   ...   ...       ...   ...   
      天使轮    NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN   NaN   
      战略投资   NaN   0.0   NaN   NaN   NaN   NaN   NaN       NaN   NaN   
      新三板    NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN   NaN   
      股权转让   NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN   NaN   
      被收购    NaN   NaN   NaN   NaN   NaN   NaN   NaN       NaN   NaN   

企业编号             1010   ...     3991     3992  3993    3994     3995  3996  \
      轮次                ...                                                  
$投资金额 A轮          NaN   ...      NaN   2500.0   NaN   800.0  17000.0   NaN   
      B轮          NaN   ...      NaN  18700.0   NaN   949.0   3100.0   NaN   
      C轮          NaN   ...      NaN      NaN   NaN  3320.0      NaN   NaN   
      IPO         0.0   ...      0.0      0.0   0.0     0.0      0.0   0.0   
      上市          NaN   ...      NaN      NaN   NaN     NaN      NaN   NaN   
...               ...   ...      ...      ...   ...     ...      ...   ...   
      天使轮         NaN   ...      NaN      NaN   NaN     NaN      NaN   NaN   
      战略投资  2923600.0   ...      0.0      0.0   NaN     0.0      NaN   NaN   
      新三板         NaN   ...      NaN      NaN   NaN     NaN      NaN   NaN   
      股权转让        NaN   ...      NaN      NaN   NaN     NaN      NaN   NaN   
      被收购         NaN   ...      NaN      NaN   NaN     NaN      NaN   NaN   

企业编号        3997  3998  3999     4000  
      轮次                               
$投资金额 A轮     NaN   NaN   NaN    585.0  
      B轮     NaN   NaN   NaN      0.0  
      C轮     NaN   NaN   NaN      NaN  
      IPO    0.0   0.0   0.0  32200.0  
      上市     NaN   NaN   NaN      NaN  
...          ...   ...   ...      ...  
      天使轮    NaN   NaN   NaN      NaN  
      战略投资   0.0   NaN   NaN      NaN  
      新三板    NaN   NaN   NaN      NaN  
      股权转让   NaN   NaN   NaN      NaN  
      被收购    NaN   NaN   NaN      NaN  

[11 rows x 3000 columns]

In [16]:
result.to_pickle('融资信息.pickle')